In [2]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

In [4]:
llm = GooglePalm(google_api_key = api_key, temperature = 0.3)

In [5]:
poem = llm("write a poem about friend")
print(poem)

**Friend**

A friend is a gift,
A treasure to behold,
A shoulder to cry on,
A heart that's always told.

A friend is a confidant,
A keeper of secrets,
A source of laughter,
A shoulder to relax on.

A friend is a supporter,
A cheerleader,
A motivator,
A shoulder to stand on.

A friend is a gift,
A treasure to behold,
A blessing to life,
A shoulder to always hold.


In [6]:
text = llm("Write a paragraph about transfomer architecture in NLP.")
print(text)

Transformer architecture is a type of neural network that has been shown to achieve state-of-the-art results on a variety of natural language processing tasks. It is based on the idea of attention, which allows the model to focus on different parts of the input sequence at different times. This makes it well-suited for tasks such as machine translation and text summarization, where the model needs to understand the overall meaning of a sentence in order to produce a correct output. Transformer models have become increasingly popular in recent years, and they are now used in a wide variety of NLP applications.


### Connect with database and solve some basic problems

In [7]:
from langchain.utilities import SQLDatabase

In [8]:
db_user = "root"
db_password = "mysql123"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [9]:
# create SQLDatabaseChain to interact with the database
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
ques1 = db_chain("How many t-shirts do we have left for nike in medium size and while color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in medium size and while color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'M'
SQLResult: [(29,)]
Answer:29
> Finished chain.


In [10]:
ques2 = db_chain("count the total number of Levi t-shirts in the database")



> Entering new SQLDatabaseChain chain...
count the total number of Levi t-shirts in the database
SQLQuery:SELECT count(*) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(16,)]
Answer:16
> Finished chain.


In [11]:
ques2

{'query': 'count the total number of Levi t-shirts in the database',
 'result': '16'}

In [12]:
ques3 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")
ques3



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('387'),)]
Answer:387
> Finished chain.


'387'

We get wrong answer. It should be SUM(price*stock_quantity). LLM should be made aware that it is not total price but price per unit. We will achieve this by doing few-shot learning which teaches AI models to learn from only a small number of labeled training data.

In [13]:
ques3 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('19594'),)]
Answer:19594
> Finished chain.


The LLM assumes the presence of 'start_date' and 'end_date' columns within the 'discount' table. It requires explicit instructions to only consider these columns if they exist, and otherwise, disregard them. We will do this using few shot learning.

In [ ]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

ques4 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('25223.300000'),)]
Answer:25223.3
> Finished chain.


In [ ]:
# If all Levi t-shirts are sold, how much money will be generated?

ques5 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('25887'),)]
Answer:25887
> Finished chain.


In [ ]:
ques6 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(13,), (71,)]
Answer:71
> Finished chain.


The correct SQL query: SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White';

In [ ]:
ques6 = db_chain.run("SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('84'),)]
Answer:84
> Finished chain.


In [ ]:
qns1 = db_chain.run("How many t-shirts do we have left for Levi in S size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for Levi in S size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White' AND size = 'S'
SQLResult: [(13,)]
Answer:13
> Finished chain.


In [ ]:
qns1 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' AND size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' AND size = 'S'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' AND size = 'S'
SQLResult: [(Decimal('13'),)]
Answer:13
> Finished chain.


### Few Shot Learning

##### Provide question and query pair where LLM was confused.

In [15]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Levi in S size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' AND size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "13"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "19594"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "25223.3"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "25887"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "84"
     }
]

Now, we will use HuggingFace to generate embeddings.

In [16]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

In [17]:
to_vectorize

["How many t-shirts do we have left for Levi in S size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' AND size = 'S' Result of the SQL query 13",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 19594",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 25223.3",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' Re

In [18]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, few_shots)

In [19]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '13',
  'Question': 'How many t-shirts do we have left for Levi in S size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' AND size = 'S'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '84',
  'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

In [20]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [21]:
print(_mysql_prompt)
mysql_prompt = _mysql_prompt

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [22]:
# setting up PromptTemplate using input variables

from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [23]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [24]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [25]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('84'),)]
Answer:84
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '84'}

In [26]:
new_chain("How much is the price of the inventory for all small-size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small-size t-shirts?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('19594'),)]
Answer:19594
> Finished chain.


{'query': 'How much is the price of the inventory for all small-size t-shirts?',
 'result': '19594'}